In [20]:
#添加路径
import os
import sys
'''
sys.path.append('/home/tione/notebook/LightGBM/python-package/')
sys.path.append('/opt/spark-2.4.5-bin-hadoop2.7/python')
sys.path.append('/home/tione/notebook')
sys.path.append('/opt/conda/envs/JupyterSystemEnv/lib/python36.zip')
sys.path.append('/opt/conda/envs/JupyterSystemEnv/lib/python3.6')
sys.path.append('/opt/conda/envs/JupyterSystemEnv/lib/python3.6/lib-dynload')
sys.path.append('/home/tione/.local/lib/python3.6/site-packages')
sys.path.append('/opt/conda/envs/JupyterSystemEnv/lib/python3.6/site-packages')
sys.path.append('/opt/conda/envs/JupyterSystemEnv/lib/python3.6/site-packages/xgboost')
'''
ko = ['', '/opt/spark-2.4.5-bin-hadoop2.7/python', '/home/tione/notebook', '/opt/conda/envs/JupyterSystemEnv/lib/python36.zip', 
      '/opt/conda/envs/JupyterSystemEnv/lib/python3.6', '/opt/conda/envs/JupyterSystemEnv/lib/python3.6/lib-dynload',
      '/home/tione/.local/lib/python3.6/site-packages', '/opt/conda/envs/JupyterSystemEnv/lib/python3.6/site-packages']
for i in ko:
    sys.path.append(i)

#数据分析，特征工程等
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
#模型选择，辅助
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split

#建模
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor 
from sklearn.tree import DecisionTreeRegressor


import lightgbm as lgb
import xgboost as xgb
import catboost as cb

In [21]:
train_data = pd.read_csv('eda_files/train_data_eda_0820.csv')
test_data = pd.read_csv('eda_files/test_data_eda_0820.csv')

In [22]:
pred_data = pd.read_csv('original_data/sss.csv')
pred_out_temp = test_data['outtemp']

In [23]:
train_ = train_data[0:18832]
train_y_ = train_['gaptemp']
del train_['gaptemp']
train_x_ = train_.values

test_ = train_data[18832:len(train_data)-1]
test_y_ = test_['gaptemp']
del test_['gaptemp']
test_x_ = test_.values

train_x_spl_ = train_x_[0:15620]
train_y_spl_ = train_y_[0:15620]
test_x_spl_ = train_x_[15620:len(train_x_)]
test_y_spl_ = train_y_[15620:len(train_x_)]



train_y = train_data['gaptemp']
del train_data['gaptemp']
train_x = train_data.values
test_x = test_data.values

模型处

In [20]:
model_lr = LinearRegression(normalize = True,copy_X = True, n_jobs = 4)

In [9]:
model_lasso = Lasso(normalize=True, precompute=True, warm_start=True)

In [15]:
param_lasso ={'alpha':[x/100 for x in range(1,11)]}

In [16]:
grid_lasso = GridSearchCV(model_lasso, param_lasso, cv=5, scoring="neg_mean_squared_error")

In [17]:
grid_lasso.fit(train_x,train_y)

GridSearchCV(cv=5,
             estimator=Lasso(normalize=True, precompute=True, warm_start=True),
             param_grid={'alpha': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07,
                                   0.08, 0.09, 0.1]},
             scoring='neg_mean_squared_error')

In [18]:
grid_lasso.best_params_

{'alpha': 0.01}

In [19]:
grid_lasso.best_score_

-0.6412575609777279

In [121]:
model_lasso = Lasso(alpha = 0.0001,normalize=True, precompute=True, warm_start=True)

In [122]:
model_lasso.fit(train_x,train_y)

Lasso(alpha=0.0001, normalize=True, precompute=True, warm_start=True)

In [123]:
model_lasso.intercept_

-1.1167697619198977

In [127]:
tt = [x for x in model_lasso.coef_]

In [128]:
col_name = [x for x in train_data.columns]

In [129]:
for i in range(0,len(col_name)):
    if tt[i] == 0:
        pass
    else:
        col_import.append(col_name[i])

In [143]:
train_data_s = train_data[col_import]
test_data_s = test_data[col_import]

In [146]:
train_x = train_data_s.values
test_x = test_data_s.values

In [151]:
train_x_ =  train_data_s[0:18832].values
test_x_ =  train_data_s[18832:len(train_data_s)-1].values

In [149]:
len(test_x_)

5975

In [150]:
len(test_y_)

5974

In [141]:
model_ridge = Ridge(normalize=True,)

In [158]:
result = []
for i in tqdm(range(90,101,1)):
    model_ridge = Ridge(alpha = i/100,normalize=True,random_state=2020)
    model_ridge.fit(train_x_,train_y_)
    tt = model_ridge.predict(test_x_)
    result_temp = mean_squared_error(test_y_,tt)
    result.append(result_temp)

100%|██████████| 11/11 [00:00<00:00, 138.64it/s]


In [161]:
model_ridge = Ridge(alpha = 0.97,normalize=True,random_state=2020)
model_ridge.fit(train_x_,train_y_)
tt = model_ridge.predict(test_)
result_temp = mean_squared_error(test_y_,tt)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 28 is different from 1286)

In [25]:
result_temp

0.048913993460339714

In [162]:
model_ridge_best = Ridge(alpha = 0.97,normalize=True,random_state=2020)

In [163]:
model_ridge_best.fit(train_x,train_y)

Ridge(alpha=0.97, normalize=True, random_state=2020)

In [164]:
res_model_ridge = model_ridge_best.predict(test_x)

In [167]:
pred_data['temperature'] = pred_out_temp+ res_model_ridge

In [168]:
pred_data.to_csv('upload_files/0811_ridge_eda_liuchu_lasso_select.csv',index = False)

ElasticNet回归

In [163]:
model_el = ElasticNet(normalize=True, warm_start=True)

In [169]:
param_el ={'alpha':[x/1000 for x in range(1,10,1)],'l1_ratio':[x/1000 for x in range(1,10,1)]}

In [174]:
param_el = {'alpha':[0,0.001],'l1_ratio':[0,0.001]}

In [175]:
grid_el = GridSearchCV(model_el, param_el, cv=5, scoring="neg_mean_squared_error")

SGD线性回归

In [183]:
model_sgd = SGDRegressor(penalty = 'l2', max_iter = 10000, learning_rate ='adaptive', early_stopping=True, n_iter_no_change=100)

In [184]:
params_sgd = {'alpha':[x/10 for x in range(1,11,1)]}

In [185]:
grid_sgd = GridSearchCV(model_sgd, params_sgd,cv=5, scoring="neg_mean_absolute_error")

In [186]:
grid_sgd.fit(train_x,train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=SGDRegressor(alpha=0.0001, average=False,
                                    early_stopping=True, epsilon=0.1, eta0=0.01,
                                    fit_intercept=True, l1_ratio=0.15,
                                    learning_rate='adaptive',
                                    loss='squared_loss', max_iter=10000,
                                    n_iter_no_change=100, penalty='l2',
                                    power_t=0.25, random_state=None,
                                    shuffle=True, tol=0.001,
                                    validation_fraction=0.1, verbose=0,
                                    warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                                   1.0]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_e

In [189]:
grid_sgd.best_params_

{'alpha': 0.6}

In [190]:
grid_sgd.best_score_

-720091651935334.8

In [257]:
model_tree = DecisionTreeRegressor(criterion='mse',random_state = 2020,max_depth = 8,min_samples_split = 5,min_samples_leaf = 12)

In [252]:
tt = model_tree.predict(test_x)

In [255]:
pred_data['temperature'] = pred_out_temp+ tt

In [256]:
pred_data.to_csv('upload_files/0805_tree_adjust.csv',index = False)

RF调参不太OK啊，先放下吧

In [ ]:
临时的得分 120 + 10 ：0.07648327909458953

In [61]:
result = []
for feature in tqdm([88,89,91,92]):
    for sample in tqdm([78,79,81,82]):
        model_rf = RandomForestRegressor(criterion = 'mse', n_jobs = 4,oob_score = True,random_state = 2020,n_estimators = 79,max_depth = 25,
                                        min_samples_split = 118, min_samples_leaf = 10, max_features = feature/100, max_samples = sample/100)
        model_rf.fit(train_x_,train_y_)
        tt = model_rf.predict(test_x_)
        value = mean_squared_error(test_y_,tt)
        result.append(value)

100%|██████████| 4/4 [14:45<00:00, 221.35s/it]


In [62]:
result

[0.0755167404869184,
 0.07528657473721105,
 0.0754624982619005,
 0.07731764445756768,
 0.0758617974473573,
 0.07577065068655245,
 0.07586904998709615,
 0.07635600247720274,
 0.07525975338713008,
 0.0752329107456404,
 0.0762805361361285,
 0.07641090924476729,
 0.07483980858335183,
 0.0752471391199147,
 0.0776437818476826,
 0.07634756168581164]

In [70]:
np.min(result) < 0.07419196137643615

False

In [59]:
临时最佳： 0.9 + 0.8： 0.07419196137643615

True

In [69]:
model_rf = RandomForestRegressor(criterion = 'mse', n_jobs = 4,oob_score = True,random_state = 2020,n_estimators = 79,max_depth = 25,
                                        min_samples_split = 118, min_samples_leaf = 10, max_features = 0.9, max_samples = 0.8)
model_rf.fit(train_x_,train_y_)
tt = model_rf.predict(test_x_)
mean_squared_error(test_y_,tt)

0.07419196137643615

In [79]:
model_rf_best = RandomForestRegressor(criterion = 'mse', n_jobs = 4,oob_score = True,random_state = 2020,n_estimators = 100,max_depth = 25,
                                        min_samples_split = 118, min_samples_leaf = 10, max_features = 0.9, max_samples = 0.8)

In [80]:
model_rf_best.fit(train_x,train_y)

RandomForestRegressor(max_depth=25, max_features=0.9, max_samples=0.8,
                      min_samples_leaf=10, min_samples_split=118, n_jobs=4,
                      oob_score=True, random_state=2020)

In [81]:
tt = model_rf_best.predict(test_x)

In [82]:
pred_data['temperature'] = pred_out_temp+ tt

In [83]:
np.min(tt)

-2.982069787814455

In [84]:
np.max(tt)

1.1478166791163564

In [77]:
pred_data.to_csv('upload_files/0807_rf_adjust.csv',index = False)

In [85]:
model_rf_best.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 25,
 'max_features': 0.9,
 'max_leaf_nodes': None,
 'max_samples': 0.8,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 10,
 'min_samples_split': 118,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': 4,
 'oob_score': True,
 'random_state': 2020,
 'verbose': 0,
 'warm_start': False}

In [65]:
model_rf_best.fit(train_x,train_y)

/opt/conda/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/_forest.py:832: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


RandomForestRegressor(max_depth=15, max_features=0.93, min_samples_split=13,
                      n_estimators=9, n_jobs=4, oob_score=True,
                      random_state=2020)

In [66]:
tt = model_rf_best.predict(test_x)


In [70]:
pred_data['temperature'] = pred_out_temp+ tt

In [71]:
pred_data.to_csv('upload_files/0806_rf_adjust.csv',index = False)

In [5]:
#train_data_lgb = lgb.Dataset(train_x_, train_y_, silent=True)
train_data_lgb = lgb.Dataset(train_x_, train_y_, silent=True)
test_data_lgb = lgb.Dataset(test_x_, test_y_, silent=True)

In [53]:
model_lgb = lgb.LGBMRegressor(n_jobs = 4,objective='regression',num_leaves=50,
                              learning_rate=0.02, n_estimators=100, max_depth=7,
                              metric='rmse', bagging_fraction = 0.8,feature_fraction = 0.8)

In [96]:
train_loss = []
test_loss = []
for i in tqdm([0.4,0.5,0.6,0.8,0.9]):
    model_lgb = lgb.LGBMRegressor(n_jobs = 4,objective='regression',num_leaves=5,
                              learning_rate=0.02, n_estimators=100, max_depth=5,
                              metric='rmse', feature_fraction = 0.6)
    model_lgb.fit(train_x_,train_y_)
    tt = model_lgb.predict(train_x_)
    train_loss.append(mean_squared_error(train_y_,tt))
    tt = model_lgb.predict(test_x_)
    test_loss.append(mean_squared_error(test_y_,tt))

100%|██████████| 5/5 [00:34<00:00,  6.98s/it]


In [97]:
test_loss

[0.05875854475028237,
 0.058312899936891954,
 0.05823495236680633,
 0.05880070455621831,
 0.05934914180786681]

In [115]:
model_lgb = lgb.LGBMRegressor(n_jobs = 4,objective='regression',num_leaves=5,
                              learning_rate=0.02, n_estimators=135, max_depth=5,
                              metric='rmse', feature_fraction = 0.6)

In [118]:
model_lgb.fit(train_x_,train_y_)

LGBMRegressor(feature_fraction=0.6, learning_rate=0.02, max_depth=5,
              metric='rmse', n_estimators=135, n_jobs=4, num_leaves=5,
              objective='regression')

In [119]:
tt = model_lgb.predict(test_x_)

In [120]:
mean_squared_error(test_y_,tt)

0.05501739179807562

In [121]:
model_lgb = lgb.LGBMRegressor(n_jobs = 4,objective='regression',num_leaves=5,
                              learning_rate=0.02, n_estimators=10000, max_depth=5,
                              metric='rmse', feature_fraction = 0.6)

In [122]:
model_lgb.fit(train_x_,train_y_)

LGBMRegressor(feature_fraction=0.6, learning_rate=0.02, max_depth=5,
              metric='rmse', n_estimators=10000, n_jobs=4, num_leaves=5,
              objective='regression')

In [123]:
tt = model_lgb.predict(test_x_)

In [124]:
mean_squared_error(test_y_,tt)

0.059036990304471036

lightgbm开始


In [10]:
params = {
    'boosting_type': 'gbdt', 
    'objective': 'regression', 
 
    'learning_rate': 0.1, 
    'num_leaves': 50, 
    'max_depth': 6,
 
    'subsample': 0.8, 
    'colsample_bytree': 0.8, 
    'n_jobs':12
}



In [11]:
train_data_lgb = lgb.Dataset(train_x, train_y, silent=True)

In [12]:
cv_results = lgb.cv(
    params, train_data_lgb, num_boost_round=10000, nfold=5, stratified=False, shuffle=True, metrics='mse',
    early_stopping_rounds=100, verbose_eval= 10, seed=0)



[10]	cv_agg's l2: 0.102237 + 0.00176824
[20]	cv_agg's l2: 0.0423857 + 0.000660977
[30]	cv_agg's l2: 0.0307512 + 0.000682574
[40]	cv_agg's l2: 0.0262331 + 0.000788484
[50]	cv_agg's l2: 0.0241924 + 0.000644872
[60]	cv_agg's l2: 0.0230091 + 0.000636154
[70]	cv_agg's l2: 0.0220968 + 0.000642395
[80]	cv_agg's l2: 0.0212163 + 0.00058788
[90]	cv_agg's l2: 0.0204543 + 0.000577021
[100]	cv_agg's l2: 0.0196907 + 0.000563257
[110]	cv_agg's l2: 0.0191004 + 0.000630642
[120]	cv_agg's l2: 0.0185557 + 0.000616728
[130]	cv_agg's l2: 0.0180354 + 0.000641311
[140]	cv_agg's l2: 0.0175856 + 0.000589975


KeyboardInterrupt: 

In [21]:
len(cv_results['l2-mean'])

1000

In [28]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=50,
                              learning_rate=0.1, n_estimators=1000, max_depth=7,
                              metric='rmse', bagging_fraction = 0.8,feature_fraction = 0.8)
params_test1={
    'num_leaves':range(45,60,5)
}
gsearch1 = GridSearchCV(estimator=model_lgb ,param_grid=params_test1, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=8)

In [29]:
gsearch1.fit(train_x,train_y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  15 out of  15 | elapsed:  6.2min finished


GridSearchCV(cv=5,
             estimator=LGBMRegressor(bagging_fraction=0.8, feature_fraction=0.8,
                                     max_depth=7, metric='rmse',
                                     n_estimators=1000, num_leaves=50,
                                     objective='regression'),
             n_jobs=8, param_grid={'num_leaves': range(45, 60, 5)},
             scoring='neg_mean_squared_error', verbose=1)

In [30]:
gsearch1.best_score_

-0.10116648682068666

In [31]:
gsearch1.best_params_

{'num_leaves': 55}

In [32]:
model_lgb = lgb.LGBMRegressor(objective='regression',
                              metric='mse',
                              learning_rate=0.1,
                              n_estimators=55,
                              max_depth=7,
                              num_leaves=47,
                              bagging_fraction = 0.8,
                              feature_fraction = 0.8)

In [59]:
params_test3={
    'min_child_samples': [18, 19, 20, 21, 22],
    'min_child_weight':[0.001, 0.002]
}


In [60]:
gsearch1 = GridSearchCV(estimator=model_lgb ,param_grid=params_test3, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)

In [61]:
gsearch1.fit(train_x,train_y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  3.5min finished


GridSearchCV(cv=5,
             estimator=LGBMRegressor(bagging_fraction=0.8, feature_fraction=0.8,
                                     max_depth=7, metric='mse', n_estimators=50,
                                     num_leaves=47, objective='regression'),
             n_jobs=4,
             param_grid={'min_child_samples': [18, 19, 20, 21, 22],
                         'min_child_weight': [0.001, 0.002]},
             scoring='neg_mean_squared_error', verbose=1)

In [64]:
gsearch1.best_score_

-0.06407093939349463

In [66]:
gsearch1.best_params_

{'min_child_samples': 20, 'min_child_weight': 0.001}

In [67]:
model_lgb = lgb.LGBMRegressor(objective='regression',
                              metric='mse',
                              learning_rate=0.1,
                              n_estimators=1000,
                              num_leaves=55,
                              bagging_fraction = 0.8,
                              feature_fraction = 0.8)

In [38]:

params_test4={
    'feature_fraction': [0.5,0.55,0.6],
    'bagging_fraction': [0.5,0.55,0.6],
}

In [39]:
gsearch1 = GridSearchCV(estimator=model_lgb ,param_grid=params_test4, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=8)

In [40]:
gsearch1.fit(train_x,train_y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  45 out of  45 | elapsed:  1.6min finished


GridSearchCV(cv=5,
             estimator=LGBMRegressor(bagging_fraction=0.8, feature_fraction=0.8,
                                     max_depth=7, metric='mse', n_estimators=55,
                                     num_leaves=47, objective='regression'),
             n_jobs=8,
             param_grid={'bagging_fraction': [0.5, 0.55, 0.6],
                         'feature_fraction': [0.5, 0.55, 0.6]},
             scoring='neg_mean_squared_error', verbose=1)

In [41]:
gsearch1.best_score_ 

-0.0871092140097952

In [42]:
gsearch1.best_params_

{'bagging_fraction': 0.5, 'feature_fraction': 0.6}

In [36]:
gsearch1.best_score_ 

-0.0871092140097952

In [37]:
gsearch1.best_params_

{'bagging_fraction': 0.5, 'feature_fraction': 0.6}

In [44]:
model_lgb = lgb.LGBMRegressor(objective='regression',
                              metric='mse',
                              learning_rate=0.1,
                              n_estimators=1000,
                              num_leaves=55,
                              bagging_fraction = 0.50,
                              feature_fraction = 0.60)

In [48]:
params_test4={'lambda_l1': [x/10 for x in range(1,11,2)],
              'lambda_l2': [x/10 for x in range(1,11,2)]
}

In [49]:
gsearch1 = GridSearchCV(estimator=model_lgb ,param_grid=params_test4, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=8)

In [50]:
gsearch1.fit(train_x,train_y)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 13.3min
[Parallel(n_jobs=8)]: Done 125 out of 125 | elapsed: 40.8min finished


GridSearchCV(cv=5,
             estimator=LGBMRegressor(bagging_fraction=0.5, feature_fraction=0.6,
                                     metric='mse', n_estimators=1000,
                                     num_leaves=55, objective='regression'),
             n_jobs=8,
             param_grid={'lambda_l1': [0.1, 0.3, 0.5, 0.7, 0.9],
                         'lambda_l2': [0.1, 0.3, 0.5, 0.7, 0.9]},
             scoring='neg_mean_squared_error', verbose=1)

In [51]:
gsearch1.best_params_

{'lambda_l1': 0.3, 'lambda_l2': 0.1}

In [52]:
gsearch1.best_score_

-0.09750534503609934

In [53]:
gsearch1.best_score_

-0.09750534503609934

In [24]:
params = {                   "objective":'regression',
                              "metric":'mse',
                              "learning_rate":0.02,
                             
                              "num_leaves":55,

                              "bagging_fraction" : 0.50,
                              "feature_fraction" : 0.60,
                              'n_jobs':12
    
}

In [25]:
cv_results = lgb.cv(
    params, train_data_lgb, num_boost_round=100000, nfold=5, stratified=False, shuffle=True, metrics='mse',
    early_stopping_rounds=1000, seed=0)

In [26]:
len(cv_results['l2-mean'])

22215

In [27]:
cv_results['l2-mean'][-1]

0.009265570940654196

In [30]:
model_lgb_002 = lgb.LGBMRegressor(objective='regression',
                              metric='mse',
                              learning_rate=0.02,
                              n_estimators=22215,                       
                              num_leaves=55,
                              bagging_fraction = 0.50,
                              feature_fraction = 0.60,
                                  n_jobs = 12
                             )

In [32]:
model_lgb_005.fit(train_x,train_y)

LGBMRegressor(bagging_fraction=0.5, feature_fraction=0.6, learning_rate=0.02,
              metric='mse', n_estimators=22215, num_leaves=55,
              objective='regression')

In [33]:
tt = model_lgb_005.predict(test_x)

In [ ]:
mean_squared_error(test_y_,tt)

In [103]:
mean_squared_error(test_y_,tt)

0.06666515147488015

In [19]:
model_lgb_005 = lgb.LGBMRegressor(objective='regression',
                              metric='mse',
                              learning_rate=0.02,
                              n_estimators=6102,
                              max_depth=7,
                              num_leaves=47,
                              min_child_samples = 20,
                              min_child_weight = 0.001,
                              bagging_fraction = 0.75,
                              feature_fraction = 0.65,
                              lambda_l1 = 0.5,
                              lambda_l2 = 1.0
                             )

In [20]:
model_lgb_005.fit(train_x,train_y)

[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75


LGBMRegressor(bagging_fraction=0.75, feature_fraction=0.65, lambda_l1=0.5,
              lambda_l2=1.0, learning_rate=0.02, max_depth=7, metric='mse',
              n_estimators=6102, num_leaves=47, objective='regression')

In [21]:
tt = model_lgb_005.predict(test_x)

In [22]:
np.min(tt)

-2.935431885465166

In [23]:
np.max(tt)

1.0698291119333414

In [24]:
pred_data['temperature'] = pred_out_temp+ tt

In [25]:
pred_data.to_csv('upload_files/0808_lgb_adjust_002.csv',index = False)

In [6]:
train_data_lgb = lgb.Dataset(train_x, train_y, silent=True)

In [15]:
params = {                   "objective":'regression',
                              "metric":'mse',
                              "learning_rate":0.02,
                             # n_estimators=50,
                              "max_depth":7,
                              "num_leaves":47,
                              "min_child_samples" : 20,
                              "min_child_weight" : 0.001,
                              "bagging_fraction" : 0.75,
                              "feature_fraction" : 0.65,
                              "lambda_l1" : 0.5,
                              "lambda_l2" : 1.0,
                              'n_jobs':12
    
}

In [16]:
cv_results = lgb.cv(
    params, train_data_lgb, num_boost_round=100000, nfold=5, stratified=False, shuffle=True, metrics='mse',verbose_eval = 50,
    early_stopping_rounds=3000,  seed=0)

[50]	cv_agg's l2: 0.228018 + 0.00511239
[100]	cv_agg's l2: 0.109756 + 0.00225015


KeyboardInterrupt: 

In [10]:
len(cv_results['l2-mean'])

13060

In [23]:
params = {                   "objective":'regression',
                              "metric":'mse',
                              "learning_rate":0.01,
                              'n_estimators':13060,
                              "max_depth":7,
                              "num_leaves":47,
                              "min_child_samples" : 20,
                              "min_child_weight" : 0.001,
                              "bagging_fraction" : 0.75,
                              "feature_fraction" : 0.65,
                              "lambda_l1" : 0.5,
                              "lambda_l2" : 1.0,
                             
    
}

In [24]:
model_lgb = lgb.LGBMRegressor(**params)

In [25]:
model_lgb.fit(train_x,train_y)

LGBMRegressor(bagging_fraction=0.75, feature_fraction=0.65, lambda_l1=0.5,
              lambda_l2=1.0, learning_rate=0.01, max_depth=7, metric='mse',
              n_estimators=13060, num_leaves=47, objective='regression')

In [27]:
tt = model_lgb.predict(test_x)

In [28]:
np.min(tt)

-2.918143924892387

In [29]:
np.max(tt)

1.0648997283734811

In [30]:
pred_data['temperature'] = pred_out_temp+ tt
pred_data.to_csv('upload_files/lgb_lr001_csdn_8.csv',index = False)

In [22]:
result = []
for n_ in tqdm(range(59,62,1)):
    xgb_params = {'eta': 0.1, 'n_estimators': n_, 'gamma': 0, 'max_depth': 6, 'min_child_weight': 1,
                    'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 1, 'reg_alpha': 0,
                    'seed': 2020}
    regress_model = xgb.XGBRegressor(**xgb_params)
    regress_model.fit(train_x_,train_y_)
    tt = regress_model.predict(test_x_)
    result.append(mean_squared_error(test_y_,tt))

100%|██████████| 3/3 [01:28<00:00, 29.46s/it]


In [26]:
result = []
for depth in tqdm([3,4,5,7]):
    xgb_params = {'eta': 0.1, 'n_estimators': 60, 'gamma': 0, 'max_depth': depth, 'min_child_weight': 1,
                    'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 1, 'reg_alpha': 0,
                    'seed': 2020}
    regress_model = xgb.XGBRegressor(**xgb_params)
    regress_model.fit(train_x_,train_y_)
    tt = regress_model.predict(test_x_)
    result.append(mean_squared_error(test_y_,tt))

100%|██████████| 4/4 [01:35<00:00, 23.99s/it]


In [32]:
np.min(result)

0.06519888240644324

In [33]:
result = []
for child_w in tqdm(range(1,11,1)):
    xgb_params = {'eta': 0.1, 'n_estimators': 60, 'gamma': 0, 'max_depth': 4, 'min_child_weight': child_w,
                    'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 1, 'reg_alpha': 0,
                    'seed': 2020}
    regress_model = xgb.XGBRegressor(**xgb_params)
    regress_model.fit(train_x_,train_y_)
    tt = regress_model.predict(test_x_)
    result.append(mean_squared_error(test_y_,tt))

100%|██████████| 10/10 [03:23<00:00, 20.35s/it]


In [37]:
np.min(result)

0.06312800065993317

In [57]:
result = []
for gamma in tqdm(range(50,53,1)):
    xgb_params = {'eta': 0.1, 'n_estimators': 60, 'gamma': 0, 'max_depth': 4, 'min_child_weight': 5,'gamma':gamma/100,
                    'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 1, 'reg_alpha': 0,
                    'seed': 2020}
    regress_model = xgb.XGBRegressor(**xgb_params)
    regress_model.fit(train_x_,train_y_)
    tt = regress_model.predict(test_x_)
    result.append(mean_squared_error(test_y_,tt))

100%|██████████| 3/3 [01:01<00:00, 20.40s/it]


In [67]:
result = []
for sample in tqdm(range(61,80,1)):
    xgb_params = {'eta': 0.1, 'n_estimators': 60, 'gamma': 0, 'max_depth': 4, 'min_child_weight': 5,'gamma':0.49,
                  'subsample': sample/100,
                    'colsample_bytree': 1, 'colsample_bylevel': 1,  'reg_lambda': 1, 'reg_alpha': 0,
                    'seed': 2020}
    regress_model = xgb.XGBRegressor(**xgb_params)
    regress_model.fit(train_x_,train_y_)
    tt = regress_model.predict(test_x_)
    result.append(mean_squared_error(test_y_,tt))

100%|██████████| 19/19 [08:05<00:00, 25.54s/it]


In [76]:
result = []
for sample in tqdm(range(51,70,1)):
    xgb_params = {'eta': 0.1, 'n_estimators': 60, 'gamma': 0, 'max_depth': 4, 'min_child_weight': 5,'gamma':0.49,'subsample': 0.76,
                   'colsample_bytree': sample/100,
                    'colsample_bylevel': 1,  'reg_lambda': 1, 'reg_alpha': 0,
                    'seed': 2020}
    regress_model = xgb.XGBRegressor(**xgb_params)
    regress_model.fit(train_x_,train_y_)
    tt = regress_model.predict(test_x_)
    result.append(mean_squared_error(test_y_,tt))

100%|██████████| 19/19 [05:11<00:00, 16.38s/it]


In [94]:
result = []
for lambda_ in tqdm(range(51,70,1)):
    xgb_params = {'eta': 0.1, 'n_estimators': 60, 'gamma': 0, 'max_depth': 4, 'min_child_weight': 5,'gamma':0.49,'subsample': 0.76,'colsample_bytree': 0.59,
                   'reg_lambda': lambda_,
                    'colsample_bylevel': 1,   'reg_alpha': 0,
                    'seed': 2020}
    regress_model = xgb.XGBRegressor(**xgb_params)
    regress_model.fit(train_x_,train_y_)
    tt = regress_model.predict(test_x_)
    result.append(mean_squared_error(test_y_,tt))

100%|██████████| 19/19 [05:07<00:00, 16.17s/it]


In [100]:
result = []
for alpha in tqdm(range(1,101,10)):
    xgb_params = {'eta': 0.1, 'n_estimators': 60, 'gamma': 0, 'max_depth': 4, 'min_child_weight': 5,'gamma':0.49,'subsample': 0.76,'colsample_bytree': 0.59,
                   'reg_lambda': 59,
                      'reg_alpha':alpha,
                    'colsample_bylevel': 1,   
                    'seed': 2020}
    regress_model = xgb.XGBRegressor(**xgb_params)
    regress_model.fit(train_x_,train_y_)
    tt = regress_model.predict(test_x_)
    result.append(mean_squared_error(test_y_,tt))

100%|██████████| 10/10 [02:43<00:00, 16.34s/it]


In [9]:
xgb_params = {'eta': 0.01, 'n_estimators': 10000, 'gamma': 0, 'max_depth': 4, 'min_child_weight':5,'gamma':0.49,'subsample': 0.76,'colsample_bytree': 0.59,
                    'reg_lambda': 59,'reg_alpha': 0, 'colsample_bylevel': 1,'seed': 2020}


In [10]:
xgb_model_best = xgb.XGBRegressor(**xgb_params)

In [53]:
valid_set = [(test_x_,test_y_)]

In [ ]:
xgb_model_best.fit(train_x_,train_y_,early_stopping_rounds = 50,eval_set = valid_set)

In [56]:
xgb_params_final = {'eta': 0.01, 'n_estimators': 445, 'gamma': 0, 'max_depth': 4, 'min_child_weight':5,'gamma':0.49,'subsample': 0.76,'colsample_bytree': 0.59,
                    'reg_lambda': 59,'reg_alpha': 0, 'colsample_bylevel': 1,'seed': 2020}


In [57]:
xgb_model_best = xgb.XGBRegressor(**xgb_params_final)

In [58]:
xgb_model_best.fit(train_x,train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.59, eta=0.01, gamma=0.49,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.00999999978, max_delta_step=0, max_depth=4,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=445, n_jobs=0, num_parallel_tree=1, random_state=2020,
             reg_alpha=0, reg_lambda=59, scale_pos_weight=1, seed=2020,
             subsample=0.76, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [59]:
tt = xgb_model_best.predict(test_x)

In [60]:
np.min(tt)

-2.5179243

In [61]:
np.max(tt)

0.8464936

In [62]:
pred_data['temperature'] = pred_out_temp+ tt

In [63]:
pred_data.to_csv('upload_files/xgb_adjust_0808.csv',index = False)

In [32]:
xgb_model_ = xgb.XGBRegressor()

In [33]:
xgb_model_.fit(train_x,train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [34]:
tt = xgb_model_.predict(test_x)

In [35]:
np.min(tt)

-2.818575

In [36]:
np.max(tt)

1.1107092

In [37]:
pred_data['temperature'] = pred_out_temp+ tt
pred_data.to_csv('upload_files/xgb_non_adjust_csdn_8.csv',index = False)

In [38]:
pred_data.to_csv('upload_files/xgb_non_adjust_csdn_8.csv',index = False)

In [78]:
model_ctb = cb.CatBoostRegressor(iterations=1500)

In [79]:
model_ctb.fit(train_x,train_y,verbose=20)

Learning rate set to 0.050793
0:	learn: 0.7034279	total: 125ms	remaining: 3m 8s
20:	learn: 0.3378718	total: 2.51s	remaining: 2m 56s
40:	learn: 0.2285099	total: 4.82s	remaining: 2m 51s
60:	learn: 0.1940251	total: 7.17s	remaining: 2m 49s
80:	learn: 0.1777303	total: 9.45s	remaining: 2m 45s
100:	learn: 0.1669095	total: 11.7s	remaining: 2m 42s
120:	learn: 0.1584057	total: 14s	remaining: 2m 39s
140:	learn: 0.1524089	total: 16.2s	remaining: 2m 36s
160:	learn: 0.1464726	total: 18.5s	remaining: 2m 34s
180:	learn: 0.1416549	total: 20.8s	remaining: 2m 31s
200:	learn: 0.1367536	total: 23.1s	remaining: 2m 29s
220:	learn: 0.1324560	total: 25.3s	remaining: 2m 26s
240:	learn: 0.1287736	total: 27.5s	remaining: 2m 23s
260:	learn: 0.1253937	total: 29.7s	remaining: 2m 21s
280:	learn: 0.1224059	total: 32.1s	remaining: 2m 19s
300:	learn: 0.1199190	total: 34.3s	remaining: 2m 16s
320:	learn: 0.1173949	total: 36.6s	remaining: 2m 14s
340:	learn: 0.1153392	total: 38.8s	remaining: 2m 11s
360:	learn: 0.1134319	tot

In [80]:
tt = model_ctb.predict(test_x)

In [81]:
np.min(tt)

-2.8809497060430656

In [82]:
np.max(tt)

0.9383923676408621

In [83]:
pred_data['temperature'] = pred_out_temp+ tt
pred_data.to_csv('upload_files/qqq_1500.csv',index = False)

In [5]:
model_lgb = lgb.LGBMRegressor()

In [11]:
model_lgb.fit(train_x,train_y)

LGBMRegressor()

In [12]:
tt = model_lgb.predict(test_x)

In [10]:
mean_squared_error(test_y_,tt)

0.07657918148676764

In [14]:
get_score(tt)

0.04245732810639653 0.03657477660499528 0.05113231307880032 0.0512150078989052 0.04791717639619361
min 0.03657477660499528
mean 0.04585932041705819


(0.03657477660499528, 0.04585932041705819)

In [34]:
#验证集
train_data_1 = pd.read_csv('upload_files/0811_4_he_1.csv')#0.12688
train_data_2 = pd.read_csv('upload_files/cb_non_adjust_csdn_8.csv')#0.13033
train_data_3 = pd.read_csv('upload_files/0808_lgb_adjust_002.csv')#0.13096
train_data_4 = pd.read_csv('upload_files/0808_lgb_adjust_005.csv')#0.1323
train_data_5 = pd.read_csv('upload_files/0804_lgb_eda_8_no_adjust.csv')#0.13121
def get_score(tt):
    a = train_data_1['temperature']
    b = train_data_2['temperature']
    c = train_data_3['temperature']
    d = train_data_4['temperature']
    e = train_data_5['temperature']
    
    data_verify = tt + pred_out_temp
    aa = mean_squared_error(a,data_verify)
    bb = mean_squared_error(b,data_verify)
    cc = mean_squared_error(c,data_verify)
    dd = mean_squared_error(d,data_verify)
    ee = mean_squared_error(e,data_verify)
    print(aa,bb,cc,ee,dd)
    print('min',min(aa,bb,cc,dd,ee))
    print('mean',(aa+bb+cc+dd+ee)/5)
    return min(aa,bb,cc,dd,ee),(aa+bb+cc+dd+ee)/5

In [35]:
get_score(tt)

0.08208857397131823 0.06907678803553581 0.09355706993374883 0.09279199378311932 0.08993840568617693
min 0.06907678803553581
mean 0.08549056628197983


(0.06907678803553581, 0.08549056628197983)

In [36]:
model_lgb_002 = lgb.LGBMRegressor(objective='regression',
                              metric='mse',
                              learning_rate=0.1,
                              n_estimators=1000,                       
                              num_leaves=55,
                              bagging_fraction = 0.50,
                              feature_fraction = 0.60,
                                  n_jobs = 12
                             )

In [37]:
model_lgb_002.fit(train_x,train_y)

LGBMRegressor(bagging_fraction=0.5, feature_fraction=0.6, metric='mse',
              n_estimators=1000, n_jobs=12, num_leaves=55,
              objective='regression')

In [38]:
tt = model_lgb_002.predict(test_x)

In [39]:
get_score(tt)

0.07620697112723501 0.06367854651580135 0.08757477897205691 0.08652738805856657 0.08405713251582335
min 0.06367854651580135
mean 0.07960896343789664


(0.06367854651580135, 0.07960896343789664)

In [ ]:
讯飞开发者大赛
